## Exporting the results to PDF

Once you complete the assignments, export the entire notebook as PDF and attach it to your homework solutions. 
The best way of doing that is
1. Run all the cells of the notebook.
2. Export/download the notebook as PDF (File -> Download as -> PDF via LaTeX (.pdf)).
3. Concatenate your solutions for other tasks with the output of Step 2. On linux, you can use `pdfunite`, there are similar tools for other platforms, too. You can only upload a single PDF file to Moodle.

Make sure you are using `nbconvert` version 5.5 or later by running `jupyter nbconvert --version`. Older versions clip lines that exceed page width, which makes your code harder to grade.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.datasets import load_sample_image

%matplotlib inline

def compare_images(img, img_compressed, k):
    """Show the compressed and uncompressed image side by side.
    """
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 12))
    axes[0].set_axis_off()
    if isinstance(k, str):
        axes[0].set_title(k)
    else:
        axes[0].set_title(f"Compressed to {k} colors")
    axes[0].imshow(img_compressed)
    axes[1].set_axis_off()
    axes[1].set_title("Original")
    axes[1].imshow(img)

# K-Means

In this first section you will implement the image compression algorithm from Bishop, chapter 9.1.1. Take an RGB image $X \in \mathbb{R}^{h \times w \times 3}$ and interpret it as a data matrix $X \in \mathbb{R}^{N \times 3}$. Now apply $k$-means clustering to find $k$ colors that describe the image well and replace each pixel with its associated cluster.

In [ ]:
# Alternatively try china.jpg
X = load_sample_image("flower.jpg")

# or load your own image
# X = np.array(Image.open("/home/user/path/to/some.jpg"))

In [ ]:
def kmeans(X, k):
    """Compute a k-means clustering for the data X.
    
    Parameters
    ----------
    X : np.array of size N x D
        where N is the number of samples and D is the data dimensionality
    k : int
        Number of clusters
        
    Returns
    -------
    mu : np.array of size k x D
        Cluster centers
    z : np.array of size N
        Cluster indicators, i.e. a number in 0..k - 1, for each data point in X
    """
    
    # TODO: Compute mu and z
    
    ### BEGIN SOLUTION
    N, D = X.shape
    
    # Pick k random points to initialize mu
    mu = X[np.random.choice(N, size=k, replace=False)].astype(np.float)
    
    z = np.empty(N, dtype=np.int)
    J_prev = None
    while True:
        # Update the cluster indicators
        z = ((X[:, np.newaxis] - mu[np.newaxis])**2).sum(axis=-1).argmin(axis=-1)
        
        # Count data points per cluster
        N_k = np.bincount(z, minlength=k)
        
        # Restart if we lost any clusters
        if np.any(N_k == 0):
            mu = X[np.random.choice(N, size=k, replace=False)]
            J_prev = None
            continue
            
        # Update centroids
        for i in range(k):
            mu[i] = X[z == i].sum(axis=0)
        mu = mu / N_k[:, np.newaxis]
        
        # Recompute the distortion
        J = ((X - mu[z])**2).sum()
        
        # Check for convergence
        if J_prev is None or J < J_prev:
            J_prev = J
        else:
            break
    ### END SOLUTION
    
    return mu, z

In [ ]:
# Cluster the color values
k = 5
mu, z = kmeans(X.reshape((-1, 3)), k)

# Replace each pixel with its cluster color
X_compressed = mu[z].reshape(X.shape).astype(np.uint8)

# Show the images side by side
compare_images(X, X_compressed, k)

# Gaussian Mixture Models & EM

Now you will repeat the same exercise with GMMs.

In [ ]:
def gmm_log_probability(X, pi, mu, sigma):
    """Compute the joint log-probabilities for each data point and component.
    
    Parameters
    ----------
    X : np.array of size N x D
        where N is the number of samples and D is the data dimensionality
    pi : np.array of size k
        Prior weight of each component
    mu : np.array of size k x D
        Mean vectors of the k Gaussian component distributions
    sigma : np.array of size k x D x D
        Covariance matrices of the k Gaussian component distributions
    
    Returns
    -------
    P : np.array of shape N x k
        P[i, j] is the joint log-probability of data point i under component j
    """
    
    # TODO: Compute P
    
    ## BEGIN SOLUTION
    # Precompute the log weights
    log_pi = np.log(pi)
    
    # The constant factor in the normal distribution PDF
    k = pi.shape[0]
    consts = -k / 2*np.log(2*np.pi)
    
    # Precompute the matrix log determinants and inverses
    log_det = np.log(np.linalg.det(sigma))
    sigma_inv = np.linalg.inv(sigma)
    
    # Find the pairwise distance between any data point and cluster center
    d = X[:, np.newaxis] - mu[np.newaxis]
    
    P = log_pi + consts - log_det / 2 - (d[..., np.newaxis, :] @ sigma_inv @ d[..., np.newaxis]).squeeze() / 2
    ## END SOLUTION
    
    return P
    

def em(X, k, tol=0.001):
    """Fit a Gaussian mixture model with k components to X.
    
    Parameters
    ----------
    X : np.array of size N x D
        where N is the number of samples and D is the data dimensionality
    k : int
        Number of clusters
    tol : float
        Converge when the increase in the mean of the expected joint log-likelihood
        is lower than this
        
    The algorithm should stop when the relative improvement in the optimization
    objective is less than rtol.
    
    Returns
    -------
    pi : np.array of size k
        Prior weight of each component
    mu : np.array of size k x D
        Mean vectors of the k Gaussian component distributions
    sigma : np.array of size k x D x D
        Covariance matrices of the k Gaussian component distributions
    """
    
    # TODO: Compute pi, mu, sigma
    
    ## BEGIN SOLUTION
    N, D = X.shape
    
    # Initialize the parameters
    pi = np.full(k, 1 / k)
    mu = X[np.random.choice(N, size=k, replace=False)].astype(np.float)
    sigma = np.identity(D)[np.newaxis].repeat(k, axis=0)
    
    log_probability = gmm_log_probability(X, pi, mu, sigma)
    J_prev = None
    einpath = None
    while True:
        # E step
        # Exp-normalize trick
        z = np.exp(log_probability - log_probability.max(axis=1, keepdims=True))
        z = z / z.sum(axis=1, keepdims=True)
        
        # M step
        N_k = z.sum(axis=0)
        pi = N_k / N
        mu = z.T @ X / N_k[:, np.newaxis]
        d = X[:, np.newaxis] - mu[np.newaxis]
        
        # Cache the optimal way to compute sigma
        if einpath is None:
            einpath = np.einsum_path("nkd,nkD,nk,k->kdD", d, d, z, 1 / N_k, optimize="optimal")[0]
        sigma = np.einsum("nkd,nkD,nk,k->kdD", d, d, z, 1 / N_k, optimize=einpath)
        
        # Compute the mean expected joint log-probability
        log_probability = gmm_log_probability(X, pi, mu, sigma)
        J = (log_probability * z).sum(axis=1).mean()
        
        # Check for convergence
        if J_prev is None or J - J_prev > tol:
            J_prev = J
        else:
            break
    ## END SOLUTION
    
    return pi, mu, sigma

In [ ]:
# Fit the GMM
k = 5
pi, mu, sigma = em(X.reshape((-1, 3)), k)

# Determine the most likely cluster of each pixel
log_p = gmm_log_probability(X.reshape((-1, 3)), pi, mu, sigma)
z = log_p.argmax(axis=1)

# Replace each pixel with its cluster mean
X_compressed = mu[z].reshape(X.shape).astype(np.uint8)

# Show the images side by side
compare_images(X, X_compressed, k)

# Sampling Unseen Datapoints

You have trained a generative model which allows you to sample from the learned distribution. In this section, you sample new images.

In [ ]:
def gmm_sample(N, pi, mu, sigma):
    """Sample N data points from a Gaussian mixture model.
    
    Parameters
    ----------
    N : int
        Number of data points to sample
    pi : np.array of size k
        Prior weight of each component
    mu : np.array of size k x D
        Mean vectors of the k Gaussian component distributions
    sigma : np.array of size k x D x D
        Covariance matrices of the k Gaussian component distributions
    
    Returns
    -------
    X : np.array of shape N x D
    """
    
    # TODO: Sample X
    
    ## BEGIN SOLUTION
    k, D = mu.shape
    z = np.random.choice(k, size=N, replace=True, p=pi)
    X = np.empty((N, D))
    for i in range(k):
        filter_i = z == i
        n_i = np.count_nonzero(filter_i)
        X[filter_i] = np.random.multivariate_normal(mu[i], sigma[i], size=n_i)
    ## END SOLUTION
    
    return X

In [ ]:
# Sample pixels and reshape them into the size of the original image
X_sampled = gmm_sample(np.prod(X.shape[:-1]), pi, mu, sigma).reshape(X.shape).astype(np.uint8)

# Compare the original and the sampled image
compare_images(X, X_sampled, "Sampled")

Explain what you see in the generated images. (1-3 sentences)

## BEGIN SOLUTION
A GMM assumes that the data points are independent and identically distributed. Therefore, a pixel grid sampled from a GMM will look like noise (with a comparable color distribution to the training image) because in natural images values between adjacent pixel are strongly correlated.
## END SOLUTION